In [ ]:
import numpy as np
import numpy.random as nrand
import matplotlib.pyplot as plt
import pandas as pd
import itertools
import copy
import pickle
import multiprocessing
from scipy import special

$y=\sum_{i}^{L}\beta_ix_i+\sum_{i<j}\beta_{ij}x_ix_j+\sum_{i<j<k}\beta_{ijk}x_ix_jx_k$  

$\beta_i \sim N(0,\sigma_1^2)$  
$\beta_{ij} \sim N(0,\sigma_2^2)$  
$\beta_{ijk} \sim N(0,\sigma_3^2)$  

$\sigma_1^2=\sigma^2v_1/L$  
$\sigma_2^2=\sigma^2v_2/(^{L}_{2})$  
$\sigma_3^2=\sigma^2v_3/(^{L}_{3})$

$v_1$, $v_2$, $v_3$ are fraction of total variance determined by the ith order.  
L is the number of sites

In [ ]:
wt = nrand.randint(2,size=N)
gt_lst = np.array(list(map(list, itertools.product([0, 1], repeat=N))))

In [ ]:
def get_fit(gt,b1,b2,b3):
    fit_1 = np.sum(gt*b1)
    gt_2 = np.array([])
    for i,row in enumerate(gt*gt.reshape(-1,1)):
        gt_2 = np.concatenate([gt_2,row[i+1:]])
    fit_2 = np.sum(gt_2*b2)
    gt_3 = np.array([])
    gt_33 = gt.reshape(-1,1,1)*gt.reshape(1,-1,1)*gt.reshape(1,1,-1)
    for pos_1 in range(N-2):
        for pos_2 in range(pos_1+1,N-1):
            gt_3 = np.concatenate([gt_3,gt_33[pos_1,pos_2,pos_2+1:]])
    fit_3 = np.sum(gt_3*b3)
    return fit_1+fit_2+fit_3

def get_fitness(gt_lst,b1,b2,b3):
    fitness_lst = np.array([get_fit(gt,b1,b2,b3) for gt in gt_lst])
    MIN = np.min(fitness_lst)
    MAX = np.max(fitness_lst)
    fitness_lst = (fitness_lst - MIN) / (MAX - MIN)
    return fitness_lst



In [ ]:
# NK landscape parameters -----------------------------------------
# Change parameters to get fitness landscape of different variable site.

N = 10  # number of variable site


In [ ]:
def normalize(array):
    MAX = np.max(array)
    MIN = np.min(array)
    return (array - MIN)/(MAX - MIN)

In [ ]:
Polynomial_landscape_list = {i:[] for i in range(1,101)}
v1_loop = itertools.cycle(np.linspace(0.95,0.05,20))
for i in range(1,101):
    for j in range(10):
        Landscape_data = []
        v1 = next(v1_loop)
        v2 = (1-v1)/2; v3 = (1-v1)/2 
        b1 = nrand.normal(scale=v1,size=N)
        b2 = nrand.normal(scale=v2,size=special.comb(N,2).astype(int))
        b3 = nrand.normal(scale=v3,size=special.comb(N,3).astype(int))
        fitness_lst = get_fitness(gt_lst,b1,b2,b3)
        fitness_landscape = np.concatenate((gt_lst,fitness_lst.reshape([-1,1])),axis=1)
        fitness_landscape = normalize(fitness_landscape)
        Polynomial_landscape_list[i].append(fitness_landscape)
        print(f"{i}:{j}     ",end='\r')

In [ ]:
with open(f'../FL_data_100X10/Polynomial_{N}_landscape_list_100X10.pkl','wb') as f:
    pickle.dump(Polynomial_landscape_list,f)